In [1]:
pip install overpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import overpy
import folium
import random

# Initialize the Overpass API
api = overpy.Overpass()

# Define the Overpass query to get ports and harbors in Denmark
query = """
area["name"="Danmark"]->.boundaryarea;
(
    node(area.boundaryarea)["harbour"];
);
out center;
"""

# Fetch the data from OSM using the Overpass API
result = api.query(query)

# Initialize a folium map centered around Denmark
denmark_map = folium.Map(location=[56.26392, 9.501785], zoom_start=7)

# Define the buffer radius (in meters)
buffer_radius = 6000  # Adjust the buffer size as necessary

# Function to generate a random color in hex format
def get_random_color():
    return "#{:06x}".format(random.randint(0, 0xFFFFFF))

# Add the ports and harbors to the map with a circular buffer (no markers)
for node in result.nodes:
    # Add a marker for the port/harbor
    folium.Circle(
        location=[node.lat, node.lon],
        radius=buffer_radius,
        color=get_random_color(),
        fill=True,
        fill_color=get_random_color(),
        fill_opacity=0.6
    ).add_to(denmark_map)
    
# Save the map to an HTML file
# denmark_map.save("denmark_ports_and_harbours_random_circles.html")

# Optionally, display the map in a Jupyter notebook (if applicable)
denmark_map


In [12]:
import psycopg as pg


# Connect to PostgreSQL/PostGIS database
connection = pg.connect(
    host="localhost",  # Replace with your database host
    port="5432",       # Replace with your database port
    dbname="aisanalysis",  # Replace with your PostGIS database name
    user="postgres",   # Replace with your username
    password="your_pass"   # Replace with your password
)

# Open a cursor to perform database operations
with connection.cursor() as cursor:

    # Check if the table exists and create if not
    create_table_query = """
    CREATE TABLE IF NOT EXISTS harbours (
        id SERIAL PRIMARY KEY,
        osm_id BIGINT UNIQUE,
        name VARCHAR(255),
        geom GEOMETRY
    );
    """
    cursor.execute(create_table_query)

    # Define an insert query
    insert_query = """
    INSERT INTO harbours (osm_id, name, geom)
    VALUES (%s, %s, ST_Point(%s, %s, 4326))
    ON CONFLICT (osm_id) DO NOTHING;
    """

    # Insert nodes
    for node in result.nodes:
        lat = node.lat
        lon = node.lon
        osm_id = node.id
        name = node.tags.get('name', 'Unnamed')  # Get name or 'Unnamed' if not available
        cursor.execute(insert_query, (osm_id, name, lon, lat))
    
# Commit the transaction
connection.commit()

# Close the connection
connection.close()

print("Harbours have been successfully inserted into the PostGIS database.")

Harbours have been successfully inserted into the PostGIS database.
